# ALTERNATE SOLUTION CORRECT OF TASK 3 USINFG DATASET CSV

# use of canny edge detection, Gradient Orientation Calculation

# use of R-Table Construction,Generalized Hough Transform,Multi-Scale and Shift Detection,Image Overlay

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from skimage.feature import canny
from scipy.ndimage import sobel

# Constants for edge detection
MIN_CANNY_THRESHOLD = 1
MAX_CANNY_THRESHOLD = 400

def generate_shifts(max_shift):
    shifts = []
    for shift_y in range(-max_shift, max_shift + 1):
        for shift_x in range(-max_shift, max_shift + 1):
            shifts.append((shift_y, shift_x))
    return shifts

def read_csv_as_paths(csv_path):
    np_paths = np.genfromtxt(csv_path, delimiter=',')
    paths = []
    for i in np.unique(np_paths[:, 0]):
        np_points = np_paths[np_paths[:, 0] == i][:, 1:]
        path = []
        for j in np.unique(np_points[:, 0]):
            points = np_points[np_points[:, 0] == j][:, 1:]
            path.append(points)
        paths.append(path)
    return paths

def compute_gradient_orientation(image):
    dx = sobel(image, axis=0, mode='constant')
    dy = sobel(image, axis=1, mode='constant')
    gradient = np.arctan2(dy, dx) * 180 / np.pi
    return gradient

def build_r_table(image, origin):
    edges = canny(image, low_threshold=MIN_CANNY_THRESHOLD,
                  high_threshold=MAX_CANNY_THRESHOLD)
    gradient = compute_gradient_orientation(edges)

    r_table = defaultdict(list)
    for (i, j), value in np.ndenumerate(edges):
        if value:
            r_table[gradient[i, j]].append((origin[0] - i, origin[1] - j))
    return r_table

def accumulate_gradients(r_table, image):
    edges = canny(image, low_threshold=MIN_CANNY_THRESHOLD,
                  high_threshold=MAX_CANNY_THRESHOLD)
    gradient = compute_gradient_orientation(edges)

    accumulator = np.zeros(image.shape)
    for (i, j), value in np.ndenumerate(edges):
        if value:
            for r in r_table[gradient[i, j]]:
                accum_i, accum_j = i + r[0], j + r[1]
                if 0 <= accum_i < accumulator.shape[0] and 0 <= accum_j < accumulator.shape[1]:
                    accumulator[accum_i, accum_j] += 1
    return accumulator

def general_hough_transform(reference_image):
    reference_point = (
        reference_image.shape[0] // 2, reference_image.shape[1] // 2)
    r_table = build_r_table(reference_image, reference_point)

    def transform(query_image):
        return accumulate_gradients(r_table, query_image)

    return transform

def find_best_match(reference_images, query_image, scales, shifts):
    best_accumulator = None
    best_position = None
    best_scale = 1
    best_shift = (0, 0)
    best_reference_image = None
    max_accumulator_value = 0

    for reference_image in reference_images:
        for scale in scales:
            scaled_reference_image = cv2.resize(
                reference_image, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)

            hough_transform = general_hough_transform(scaled_reference_image)
            accumulator = hough_transform(query_image)

            for shift_y, shift_x in shifts:
                shifted_accumulator = np.roll(accumulator, shift=(shift_y, shift_x), axis=(0, 1))
                max_value = shifted_accumulator.max()

                if max_value >= max_accumulator_value:
                    max_accumulator_value = max_value
                    best_accumulator = shifted_accumulator
                    best_position = np.unravel_index(shifted_accumulator.argmax(), shifted_accumulator.shape)
                    best_scale = scale
                    best_shift = (shift_y, shift_x)
                    best_reference_image = scaled_reference_image

    return best_position, best_scale, best_shift, best_reference_image

def overlay_reference(query_image, reference_image, position):
    ref_h, ref_w = reference_image.shape
    q_h, q_w = query_image.shape

    ref_h_half = ref_h // 2
    ref_w_half = ref_w // 2

    pos_y, pos_x = position
    start_y = max(0, pos_y - ref_h_half)
    start_x = max(0, pos_x - ref_w_half)
    end_y = min(q_h, pos_y + ref_h_half)
    end_x = min(q_w, pos_x + ref_w_half)

    ref_start_y = ref_h_half - (pos_y - start_y)
    ref_start_x = ref_w_half - (pos_x - start_x)
    ref_end_y = ref_start_y + (end_y - start_y)
    ref_end_x = ref_start_x + (end_x - start_x)

    query_image[start_y:end_y, start_x:end_x] = np.maximum(
        query_image[start_y:end_y, start_x:end_x],
        reference_image[ref_start_y:ref_end_y, ref_start_x:ref_end_x]
    )

    return query_image

def detect_shape(reference_images, query_image):
    scales = [1.0]
    shifts = [(0, 0)]

    best_position, best_scale, best_shift, best_reference_image = find_best_match(
        reference_images, query_image, scales, shifts)

    plt.clf()
    plt.gray()

    plt.title('Original Query Image')
    plt.imshow(query_image, cmap='gray')
    plt.show()

    if best_position and best_reference_image is not None:
        scaled_reference_image = cv2.resize(
            best_reference_image, None, fx=best_scale, fy=best_scale, interpolation=cv2.INTER_LINEAR)
        shifted_position = (best_position[0] + best_shift[0], best_position[1] + best_shift[1])
        final_image = overlay_reference(query_image.copy(), scaled_reference_image, shifted_position)

        plt.title('Final Image with Detected Shape')
        plt.imshow(final_image, cmap='gray')
        plt.show()

def shapes_to_image(shape_paths, image_shape=(250, 250)):
    image = np.zeros(image_shape, dtype=np.uint8)
    for shape in shape_paths:
        for points in shape:
            for x, y in points:
                image[int(y), int(x)] = 255
    return image

def test_shape_detection():
    reference_shapes_list = [
        read_csv_as_paths(r"C:\Users\arush\Downloads\single_ellipse.csv"),
        read_csv_as_paths(r"C:\Users\arush\Downloads\double_ellipse.csv")
    ]
    query_shapes = read_csv_as_paths(r"C:\Users\arush\OneDrive\Documents\problems\occlusion2.csv")

    reference_images = [shapes_to_image(shapes) for shapes in reference_shapes_list]
    query_image = shapes_to_image(query_shapes)

    detect_shape(reference_images, query_image)

def test_another_shape_detection():
    reference_shapes_list = [
        read_csv_as_paths(r"C:\Users\arush\Downloads\single_ellipse.csv"),
        read_csv_as_paths(r"C:\Users\arush\Downloads\double_ellipse.csv")
    ]
    query_shapes = read_csv_as_paths(r"C:\Users\arush\OneDrive\Documents\problems\occlusion1.csv")

    reference_images = [shapes_to_image(shapes) for shapes in reference_shapes_list]
    query_image = shapes_to_image(query_shapes)

    detect_shape(reference_images, query_image)

if __name__ == '__main__':
    test_shape_detection()
    test_another_shape_detection()
